<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata forecasting and feature extraction tools using python

in Forecastingdata, some forecasting and feature calculation tools are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [1]:
#@title
%load_ext rpy2.ipython

Clone the repository, for reproducibility. No need to clone everytime if you have already have the folder!.

In [2]:
!git clone https://github.com/pmontman/TSForecasting

fatal: destination path 'TSForecasting' already exists and is not an empty directory.


In [9]:
%cd TSForecasting/
!git pull
%cd ..

/content/TSForecasting
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 8), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/pmontman/TSForecasting
   1864540..9224864  master     -> origin/master
Updating 1864540..9224864
Fast-forward
 experiments/feature_experiments.R        |  193 +-
 experiments/feature_functions.R          |  196 ++
 experiments/fixed_horizon.R              |  273 +--
 experiments/fixed_horizon_functions.R    |  268 +++
 experiments/forecastingdata_python.ipynb | 3839 ++++++++++--------------------
 5 files changed, 1790 insertions(+), 2979 deletions(-)
 create mode 100644 experiments/feature_functions.R
 create mode 100644 experiments/fixed_horizon_functions.R
/content


We now prepare the R environment for processing: installing and loading the required packages

In [3]:
%%R
install.packages("devtools")
devtools::install_github("hendersontrent/catch22")
install.packages("tidyverse")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")
library("Rcatch22")

R[write to console]: Skipping install of 'Rcatch22' from a github remote, the SHA1 (a93d1a9b) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/tsibble_1.0.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1517998 bytes (1.4 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

Install glmnet package for linear models

In [4]:
%%R
install.packages('glmnet')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/glmnet_4.1-2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2184440 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

In [8]:
! ls TSForecasting/experiments

deep_learning_experiments.py  fixed_horizon.R		    rolling_origin.R
feature_experiments.R	      forecastingdata_python.ipynb


In [11]:
%%R
BASE_DIR <- "TSForecasting"
source("TSForecasting/experiments/feature_functions.R")

The fixed horizon we need to try line by line to skip errors

In [13]:
%%R
source("TSForecasting/experiments/fixed_horizon_functions.R")

R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v1.0.0 loaded.


R[write to console]: 
Attaching package: ‘greybox’


R[write to console]: The following object is masked from ‘package:tidyr’:

    spread


R[write to console]: This is package "smooth", v3.1.2


R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘Matrix’


R[write to console]: The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


R[write to console]: Loaded glmnet 4.1-2



## Part 2) Using tools directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [14]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']
do_fixed_horizon_local_forecasting = robjects.r['do_fixed_horizon_local_forecasting']
do_fixed_horizon_global_forecasting = robjects.r['do_fixed_horizon_global_forecasting']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [15]:
%%capture cap --no-stderr
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

R[write to console]: Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [16]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/sample_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/sample_features.csv

head of the tsfeatures features
sample_features.csv
mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,flat_spots,alpha,beta,hurst,lumpiness,nonlinearity,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,stability,unitroot_kpss,unitroot_pp,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough
105.652455128205,669.530932765715,1.2706668176004,52,0.755290510889679,2,0.959745807190754,2,0.710805797539225,1.51175743379058,-0.239716429411997,0.105249143633781,-0.539721534779089,0.372057773793558,-0.0890329975757608,0.257725362872141,22,0.795329846811295,14,0.628482857552452,0.000100053599224661,0.989518146632079,0.254142008742879,0.315470866286382,0.530751963124808,0.104820622256713,0.656220390192105,0.0363289043636947,0.160377933409522,0.459213533792908,-39.7197694054302,1,52,0.233768779821994,0.00021

Compute the local forecasts.

In [17]:
do_fixed_horizon_local_forecasting("sample", "ses", "sample.tsf", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fd0a9c148c0> [RTYPES.NILSXP]

Try the global model forecasts.

In [14]:
do_fixed_horizon_global_forecasting("sample", 25, "sample.tsf", "pooled_regression", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f35b0dcc4b0> [RTYPES.NILSXP]